In [1]:
import math
import collections
import pickle
import random
import scipy
import json

from matplotlib import pyplot as plt

import GPy
import numpy as np
import pandas as pd
from tqdm import tqdm

# from Kernel import MixtureViaSumAndProduct, CategoryOverlapKernel

In [2]:
selected = pd.read_excel('CheckConditions_Selected.xlsx')
Exp_1 = selected.values


In [3]:
main_file_path = '/Users/harini/Dropbox (MIT)/Harini Narayanan’s files/Home/Biotech/MediaOptimization/BO_MediaOptimization/Molecules/HSA/HSA_Prod_CoCaBO/'
carbon_source_filepath = '/Users/harini/Dropbox (MIT)/Harini Narayanan’s files/Home/Biotech/MediaOptimization/BO_MediaOptimization/Molecules/'


In [4]:
Stock_solid = pd.read_excel(carbon_source_filepath+'CarbonSourceInfo.xlsx', 'Stocks_solid')
Stock_liquid = pd.read_excel(carbon_source_filepath +'CarbonSourceInfo.xlsx', 'Stocks_liquid')

Carbon_Names = Stock_solid['Carbon Source'].values.tolist()
Carbon_Names.append(Stock_liquid['Carbon Source'][1])
Carbon_Names.append(Stock_liquid['Carbon Source'][2])


Carbon_Ub = [50]*17 #[g/L]
Carbon_Ub.append(10) # Glycerol [mL/L]
Carbon_Ub.append(50) #Ethanol [mL/L]

OG_Gly_Ub = 100 #[mL/L]
Met_Ub = 100 #[mL/L]


Stock_Conc = Stock_solid['g/mL'].values.tolist()
Stock_Conc.append(Stock_liquid['mL/mL'][1].tolist())
Stock_Conc.append(Stock_liquid['mL/mL'][2].tolist())

OG_Stock_Conc = Stock_liquid['mL/mL'][0].tolist()


Selected_Carbon = []
Carbon = []
OG_Gly = []
Met = []

initN = selected.shape[0]
for i in range(initN):
    Selected_Carbon.append(Carbon_Names.index(selected['Carbon_Type'].iloc[i]))
    temp_factor = Carbon_Ub[Carbon_Names.index(selected['Carbon_Type'].iloc[i])]
    temp = Exp_1[i,1]/ temp_factor # Input - (g/L)
    Carbon.append(temp)
    OG_Gly.append(Exp_1[i,2]/OG_Gly_Ub * 1000/100) #Input - (%)
    Met.append(Exp_1[i,3]/ Met_Ub * 1000/100)#Input - (%)
    
    
Selected_Carbon.append(int(0))
Carbon.append(0.8)
OG_Gly.append(0.4)
Met.append(0.15)

Experiment_1 = {'Carbon_Type': Selected_Carbon,
               'Conc_Carbon': Carbon,
               'Gly_OG': OG_Gly,
               'Met_Prod': Met}

pd.DataFrame(Experiment_1).to_csv('5_ExperimentalDesign.csv')

In [5]:
data = pd.read_csv('5_ExperimentalDesign.csv')
z_next = data.iloc[:,1:].values
batch_size = data.shape[0]

### 5. Conversion to actual experimental execution

In [6]:
Stock_solid = pd.read_excel(carbon_source_filepath+'CarbonSourceInfo.xlsx', 'Stocks_solid')
Stock_liquid = pd.read_excel(carbon_source_filepath +'CarbonSourceInfo.xlsx', 'Stocks_liquid')

Carbon_Names = Stock_solid['Carbon Source'].values.tolist()
Carbon_Names.append(Stock_liquid['Carbon Source'][1])
Carbon_Names.append(Stock_liquid['Carbon Source'][2])

Carbon_Ub = [50]*17 #[g/L]
Carbon_Ub.append(10) # Glycerol [mL/L]
Carbon_Ub.append(50) #Ethanol [mL/L]

OG_Gly_Ub = 100 #[mL/L]
Met_Ub = 100 #[mL/L]

Stock_Conc = Stock_solid['g/mL'].values.tolist()
Stock_Conc.append(Stock_liquid['mL/mL'][1].tolist())
Stock_Conc.append(Stock_liquid['mL/mL'][2].tolist())

OG_Stock_Conc = Stock_liquid['mL/mL'][0].tolist()



In [7]:
Selected_Carbon = []
Carbon_mL = []
OG_Gly_mL = []
Met_mL = []


for i in range(batch_size):
    Selected_Carbon.append(Carbon_Names[int(z_next[i,0])])
    temp_factor = Carbon_Ub[int(z_next[i,0])]/ Stock_Conc[int(z_next[i,0])]
    temp = z_next[i,1] * temp_factor * 3
    Carbon_mL.append(temp)
    OG_Gly_mL.append(z_next[i,2] * OG_Gly_Ub * 3/OG_Stock_Conc)
    Met_mL.append(z_next[i,3] * Met_Ub * 3)
    


Experiment_4_3mL = {'Carbon_Type': Selected_Carbon,
               'Conc_Carbon [uL]': Carbon_mL,
               'Gly_OG [uL]': OG_Gly_mL,
               'Met_Prod [uL]': Met_mL}

pd.DataFrame(Experiment_4_3mL).to_csv('5_ExperimentPlan_mLValue_3mL.csv')